In [1]:
import re
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# "magic" command to make plots show up in the notebook
%matplotlib inline 

In [56]:
pd.set_option('display.max_columns', 200)
pd.options.display.max_rows = 1000
# pd.options.display.max_columns = 1000
# pd.options.display.max_colwidth = 1000

In [2]:
%pwd

'/Users/a06411/Documents/GitHub/js_automl_pycaret/lending_club_da/Default Prediction_최종'

In [58]:
path_data = "/Users/a06411/Documents/data_hub/lending_club/lending_club.csv"

In [59]:
df = pd.read_csv(path_data)

/var/folders/h4/44486vyn1_xbs13g1z50l5m00000gn/T/ipykernel_92506/502107637.py:1: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_data)


In [61]:
df['loan_status'].value_counts()

Fully Paid                                             1076751
Current                                                 878317
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: loan_status, dtype: int64

In [62]:
###Drop Current
df = df[df['loan_status'] != 'Current']

In [63]:
df['loan_status'].value_counts()

Fully Paid                                             1076751
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: loan_status, dtype: int64

In [64]:
df.shape

(1382384, 151)

## Drop Columns which contains NaN value more than 90% 

In [65]:
drop_cols = df.columns[df.isnull().mean() > 0.9]

In [66]:
drop_cols

Index(['member_id', 'desc', 'next_pymnt_d', 'annual_inc_joint', 'dti_joint',
       'verification_status_joint', 'revol_bal_joint',
       'sec_app_fico_range_low', 'sec_app_fico_range_high',
       'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths',
       'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util',
       'sec_app_open_act_il', 'sec_app_num_rev_accts',
       'sec_app_chargeoff_within_12_mths',
       'sec_app_collections_12_mths_ex_med',
       'sec_app_mths_since_last_major_derog', 'hardship_type',
       'hardship_reason', 'hardship_status', 'deferral_term',
       'hardship_amount', 'hardship_start_date', 'hardship_end_date',
       'payment_plan_start_date', 'hardship_length', 'hardship_dpd',
       'hardship_loan_status', 'orig_projected_additional_accrued_interest',
       'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
       'debt_settlement_flag_date', 'settlement_status', 'settlement_date',
       'settlement_amount', 'settlement_perce

In [67]:
df = df.drop(drop_cols,axis=1)

In [68]:
df.shape

(1382384, 112)

## Drop useless columns
- Date columns (since no FE in following project), titles , zipcode (encrypted) , policy_code (single unique value)

In [69]:
df.drop(['last_pymnt_d','last_credit_pull_d','earliest_cr_line'] , 1, inplace = True)

/var/folders/h4/44486vyn1_xbs13g1z50l5m00000gn/T/ipykernel_92506/1870192760.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['last_pymnt_d','last_credit_pull_d','earliest_cr_line'] , 1, inplace = True)


In [70]:
df.shape

(1382384, 109)

In [71]:
df.drop(['url','emp_title','title','zip_code','policy_code'] , 1, inplace = True)  ## policy_code 추가(unique값 1개라 의미없)

/var/folders/h4/44486vyn1_xbs13g1z50l5m00000gn/T/ipykernel_92506/1064962860.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['url','emp_title','title','zip_code','policy_code'] , 1, inplace = True)  ## policy_code 추가(unique값 1개라 의미없)


In [72]:
df.shape

(1382384, 104)

In [73]:
obj_col = df.select_dtypes('object').columns.values

In [74]:
obj_col

array(['id', 'term', 'grade', 'sub_grade', 'emp_length', 'home_ownership',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan',
       'purpose', 'addr_state', 'initial_list_status', 'application_type',
       'hardship_flag', 'disbursement_method', 'debt_settlement_flag'],
      dtype=object)

In [75]:
df[obj_col].nunique().sort_values(ascending=False)

id                      1382384
issue_d                     139
addr_state                   51
sub_grade                    35
purpose                      14
emp_length                   11
loan_status                   8
grade                         7
home_ownership                6
verification_status           3
term                          2
pymnt_plan                    2
initial_list_status           2
application_type              2
hardship_flag                 2
disbursement_method           2
debt_settlement_flag          2
dtype: int64

## Drop rows fulled with NaN values

In [76]:
df[df['grade'].isna()].head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
421095,Total amount funded in policy code 1: 6417608175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421096,Total amount funded in policy code 2: 1944088810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528961,Total amount funded in policy code 1: 1741781700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528962,Total amount funded in policy code 2: 564202131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
651664,Total amount funded in policy code 1: 1791201400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
df = df[df['grade'].notna()]

In [78]:
df.shape

(1382351, 104)

# Label Encoding

## map 메서드로 레이블 인코딩

In [79]:
df['verification_status'].value_counts()

Source Verified    536031
Verified           429617
Not Verified       416703
Name: verification_status, dtype: int64

In [80]:
df['verification_status'] = df.verification_status.map({'Verified': 0, 'Source Verified': 1, 'Not Verified': 2})

In [81]:
df['debt_settlement_flag'].value_counts()

N    1348133
Y      34218
Name: debt_settlement_flag, dtype: int64

In [82]:
df['debt_settlement_flag'] = df.debt_settlement_flag.map({'N': 0, 'Y': 1})

In [83]:
df['initial_list_status'].value_counts()

w    810885
f    571466
Name: initial_list_status, dtype: int64

In [84]:
df['initial_list_status'] = df.initial_list_status.map({'w': 0, 'f': 1})

In [85]:
df['application_type'].value_counts()

Individual    1352767
Joint App       29584
Name: application_type, dtype: int64

In [86]:
df['application_type'] = df.application_type.map({'Individual': 0, 'Joint App': 1})

In [87]:
df['hardship_flag'].value_counts()

N    1381523
Y        828
Name: hardship_flag, dtype: int64

In [88]:
df['hardship_flag'] = df.hardship_flag.map({'N': 0, 'Y': 1})

In [89]:
df['pymnt_plan'].value_counts() 

n    1381734
y        617
Name: pymnt_plan, dtype: int64

In [90]:
df['pymnt_plan'] = df.pymnt_plan.map({'n': 0, 'y': 1})

In [91]:
df['disbursement_method'].value_counts()

Cash         1374460
DirectPay       7891
Name: disbursement_method, dtype: int64

In [92]:
df['disbursement_method'] = df.disbursement_method.map({'Cash': 0, 'DirectPay': 1})

## ML로 레이블 인코딩 처리 

In [93]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [94]:
df['addr_state'] = le.fit_transform(df['addr_state'].astype(str))
df['sub_grade'] = le.fit_transform(df['sub_grade'].astype(str))
df['purpose'] = le.fit_transform(df['purpose'].astype(str))
df['emp_length'] = le.fit_transform(df['emp_length'].astype(str))
df['home_ownership'] = le.fit_transform(df['home_ownership'].astype(str))
df['term'] = le.fit_transform(df['term'].astype(str))

In [95]:
df['loan_status'].value_counts()

Fully Paid                                             1076751
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: loan_status, dtype: int64

In [96]:
###Targets

df['grade'] = df.grade.map({'A': 0, 'B': 1,'C': 2, 'D': 3,'E': 4, 'F': 5,'G': 6})   
df['loan_status'] = df.loan_status.map({'Fully Paid':0, 
                                            'Charged Off': 1
                                            , 'Late (31-120 days)': 1
                                           , 'Late (16-30 days)': 1
                                           , 'In Grace Period': 0
                                           , 'Does not meet the credit policy. Status:Fully Paid': 0
                                           , 'Does not meet the credit policy. Status:Charged Off': 1
                                           , 'Default': 1})

In [97]:
# df.fillna(-1,inplace = True)

In [98]:
# from sklearn.model_selection import train_test_split
# train, test = train_test_split(df,test_size = 0.2, random_state = 2021)

In [99]:
# train.reset_index(drop=True, inplace=True)
# test.reset_index(drop=True, inplace=True)

In [100]:
# answer = test[['id','loan_status']]
# test.drop('loan_status',1, inplace = True)

In [101]:
# # Kernel Density Plot
# fig = plt.figure(figsize=(15,8),)
# ax=sns.kdeplot(df.loc[(df['loan_status'] == 0),'last_fico_range_high'] , color='gray',shade=True,label='Normal')
# ax=sns.kdeplot(df.loc[(df['loan_status'] == 1),'last_fico_range_high'] , color='g',shade=True, label='Default')
# plt.title('last_fico_range_high Distribution Normal vs Default', fontsize = 25, pad = 40)
# plt.ylabel("Frequency of Default", fontsize = 15, labelpad = 20)
# plt.xlabel("last_fico_range_high", fontsize = 15, labelpad = 20);

In [102]:
# # Kernel Density Plot
# fig = plt.figure(figsize=(15,8),)
# ax=sns.kdeplot(df.loc[(df['loan_status'] == 0),'last_fico_range_low'] , color='gray',shade=True,label='Normal')
# ax=sns.kdeplot(df.loc[(df['loan_status'] == 1),'last_fico_range_low'] , color='g',shade=True, label='Default')
# plt.title('last_fico_range_low Distribution Normal vs Default', fontsize = 25, pad = 40)
# plt.ylabel("Frequency of Default", fontsize = 15, labelpad = 20)
# plt.xlabel("last_fico_range_low", fontsize = 15, labelpad = 20);

In [108]:
path_write ="/Users/a06411/Documents/data_hub/lending_club/df_preprocessesd.pkl"

In [103]:
df.to_pickle(path_write)
# train.to_pickle("train.pkl")
# test.to_pickle("test.pkl")
# answer.to_pickle("answer.pkl")

In [104]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1382351 entries, 0 to 2260697
Columns: 104 entries, id to debt_settlement_flag
dtypes: float64(87), int64(15), object(2)
memory usage: 1.1+ GB


In [105]:
df.columns

Index(['id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_length',
       ...
       'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens',
       'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit', 'hardship_flag', 'disbursement_method',
       'debt_settlement_flag'],
      dtype='object', length=104)

In [106]:
df.values

array([[68407277, 3600.0, 3600.0, ..., 0, 0, 0],
       [68355089, 24700.0, 24700.0, ..., 0, 0, 0],
       [68341763, 20000.0, 20000.0, ..., 0, 0, 0],
       ...,
       ['89996426', 32000.0, 32000.0, ..., 0, 0, 0],
       ['90006534', 16000.0, 16000.0, ..., 0, 0, 0],
       ['88224441', 24000.0, 24000.0, ..., 0, 0, 1]], dtype=object)

In [107]:
df.memory_usage()

Index                             11058808
id                                11058808
loan_amnt                         11058808
funded_amnt                       11058808
funded_amnt_inv                   11058808
term                              11058808
int_rate                          11058808
installment                       11058808
grade                             11058808
sub_grade                         11058808
emp_length                        11058808
home_ownership                    11058808
annual_inc                        11058808
verification_status               11058808
issue_d                           11058808
loan_status                       11058808
pymnt_plan                        11058808
purpose                           11058808
addr_state                        11058808
dti                               11058808
delinq_2yrs                       11058808
fico_range_low                    11058808
fico_range_high                   11058808
inq_last_6m